# IBM Applied Data Science Capstone

## Segmenting and Clustering Neighborhoods in Toronto


This notebook will explore, segment, and cluster the neighborhoods in the city of Toronto. 

Data will be sourced from a Wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). 

BeautifulSoup package ( http://beautiful-soup-4.readthedocs.io/en/latest/) will be used to scrap and transform data into a dataframe for ananlysis.


***

## Question 1

In [31]:
# Import required packages.

from bs4 import BeautifulSoup
import requests
import pandas as pd


In [35]:
#Reading the data from wikipedia into an xml list.
source =requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')

table = soup.find('table')

#Create an empty Dataframe with correct headings
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

# Reading all the postcode, borough, neighborhood from the lxml and adding into dataframe
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data   

In [36]:
   df.head(12)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [37]:
# Droping cells where Borough = Not assigned

df=df[df.Borough != 'Not assigned']
df.head(12)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [38]:
# Checking for Rows with the same Postal Code 

duplicat_postal_code=df[df.duplicated(['PostalCode'])]

print(duplicat_postal_code)

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


**There are no duplicated postal Codes, no further processing required**

In [43]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(12)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [55]:
df.columns=['Postalcode','Borough','Neighborhood']
df.shape

(103, 3)

***

## Question 2

In [56]:
# Getting longitute and Latitude of each neighborhood from a csv file

!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data

coordinateDF = pd.read_csv('Toronto_long_lat_data.csv')

coordinateDF.columns=['Postalcode','Latitude','Longitude']

coordinateDF.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
# combining the neighbourhood data frame with the related longitute and latitude based on Postal code

new_df=pd.merge(df, coordinateDF[['Postalcode','Latitude', 'Longitude']], on='Postalcode')

new_df.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


***

## Question 3

## Clustering and Segementing Toronto Neighboughoods